# Exploration of Zillow Zestimates

### Hypotheses
-

In [1]:
# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import os

from matplotlib import cm
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

# wrangle
import wrangle
import env
import util

- Acquire

In [2]:
# acquires data from SQL database if it's not already locally saved as a .csv 
# file and combines requested tables into a dataframe using wrangle module
z_df = wrangle.get_zillow_data()

- Prepare

Sort the dataframe by date in ascending order then drops any duplicates that only keeps the most recent purchase for any properties sold more than once in the same year

In [3]:
# sorted dataframe by transaction date
z_df = z_df.sort_values(by='transactiondate')

# drops all but the last duplicate row by parcelid
z_df = z_df.drop_duplicates(subset=['parcelid'], keep='last')

In [4]:
# gets rid of any columns with more that 2% null values
z_df = wrangle.percentage_dropper(z_df)

Drops any remaining duplicates, columns, and rows with nulls, then converts some columns from integer to floats and purchase date to datetime datatype.

In [5]:
# Groups our functions used to clean up our data into a single function for ease of use
z_df = wrangle.clean_zillow(z_df)

In [6]:
#Adds columns with the suffix _outliers for all the numeric columns in the given dataframe
wrangle.add_upper_outlier_columns(z_df, k=1.5)

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,...,lotsizesquarefeet_outliers,rawcensustractandblock_outliers,regionidcity_outliers,regionidzip_outliers,yearbuilt_outliers,structuretaxvaluedollarcnt_outliers,taxvaluedollarcnt_outliers,landtaxvaluedollarcnt_outliers,taxamount_outliers,censustractandblock_outliers
585,14297519,0.025595,2017-01-01,3.5,4,3100,6059,33634931.0,-117869207.0,4506,...,0.0,0.000000,0.0,0.0,0,47572.375,0.0,0.0,0.0,0.000000e+00
584,17052889,0.055619,2017-01-01,1.0,2,1465,6111,34449266.0,-119281531.0,12647,...,0.0,194960.798518,0.0,0.0,0,0.000,0.0,0.0,0.0,1.949638e+11
583,14186244,0.005383,2017-01-01,2.0,3,1243,6059,33886168.0,-117823170.0,8432,...,0.0,0.000000,0.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00
582,12177905,-0.103410,2017-01-01,3.0,4,2376,6037,34245180.0,-118240722.0,13038,...,0.0,0.000000,314654.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00
581,12095076,-0.001011,2017-01-01,3.0,4,2962,6037,34145202.0,-118179824.0,63000,...,49481.5,0.000000,0.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51815,11000655,0.020615,2017-09-20,2.0,2,1286,6037,34245368.0,-118282383.0,47405,...,33886.5,0.000000,0.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00
51814,17239384,0.013209,2017-09-21,2.0,4,1612,6111,34300140.0,-118706327.0,12105,...,0.0,195034.797528,0.0,0.0,0,0.000,0.0,0.0,0.0,1.950378e+11
51813,12773139,0.037129,2017-09-21,1.0,3,1032,6037,34040895.0,-118038169.0,5074,...,0.0,0.000000,0.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00
51812,12826780,0.007204,2017-09-25,2.0,3,1762,6037,33937685.0,-117996709.0,6347,...,0.0,0.000000,0.0,0.0,0,0.000,0.0,0.0,0.0,0.000000e+00


In [7]:
# drops rows for outliers in the outlier columns from dataframe
z_df.drop(z_df.index[z_df['bathroomcnt_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['bedroomcnt_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['calculatedfinishedsquarefeet_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['lotsizesquarefeet_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['structuretaxvaluedollarcnt_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['taxvaluedollarcnt_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['landtaxvaluedollarcnt_outliers'] != 0], inplace=True)
z_df.drop(z_df.index[z_df['taxamount_outliers'] != 0], inplace=True)


In [8]:
# drops columns that originally held the outliers
z_df = z_df.drop(['parcelid_outliers', 'logerror_outliers', 'fips_outliers', 'latitude_outliers', 'longitude_outliers', 'rawcensustractandblock_outliers', 'regionidcity_outliers', 'regionidzip_outliers', 'yearbuilt_outliers', 'bathroomcnt_outliers', 'censustractandblock_outliers', 'bedroomcnt_outliers', 'calculatedfinishedsquarefeet_outliers', 'lotsizesquarefeet_outliers', 'structuretaxvaluedollarcnt_outliers', 'taxvaluedollarcnt_outliers', 
'landtaxvaluedollarcnt_outliers', 'taxamount_outliers'], axis = 1)

Checking to see if correct columns were dropped

In [9]:
z_df

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,...,rawcensustractandblock,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,county
584,17052889,0.055619,2017-01-01,1.0,2,1465,6111,34449266.0,-119281531.0,12647,...,6.111001e+07,13091,97099,1967,88000,464000,376000,5672.48,61110010023006,6111
583,14186244,0.005383,2017-01-01,2.0,3,1243,6059,33886168.0,-117823170.0,8432,...,6.059022e+07,21412,97078,1962,85289,564778,479489,6488.30,60590218022012,6059
582,12177905,-0.103410,2017-01-01,3.0,4,2376,6037,34245180.0,-118240722.0,13038,...,6.037300e+07,396551,96330,1970,108918,145143,36225,1777.51,60373001001006,6037
580,12069064,0.101723,2017-01-01,1.0,2,738,6037,34149214.0,-118239357.0,4214,...,6.037302e+07,45457,96325,1922,18890,218552,199662,2366.08,60373020041001,6037
527,12579560,-0.025721,2017-01-02,1.0,2,1027,6037,33816016.0,-118271776.0,5574,...,6.037544e+07,10723,96229,1951,27521,49034,21513,1179.11,60375437025006,6037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51929,11122390,0.012901,2017-09-19,2.0,3,1194,6037,34439229.0,-118486609.0,9080,...,6.037920e+07,54311,96374,1984,121648,294712,173064,4184.57,60379200281022,6037
51814,17239384,0.013209,2017-09-21,2.0,4,1612,6111,34300140.0,-118706327.0,12105,...,6.111008e+07,27110,97116,1964,50683,67205,16522,1107.48,61110084022016,6111
51813,12773139,0.037129,2017-09-21,1.0,3,1032,6037,34040895.0,-118038169.0,5074,...,6.037434e+07,36502,96480,1954,32797,49546,16749,876.43,60374338022005,6037
51812,12826780,0.007204,2017-09-25,2.0,3,1762,6037,33937685.0,-117996709.0,6347,...,6.037503e+07,14634,96171,1955,140000,522000,382000,6317.15,60375034021020,6037


In [10]:
z_df.head(1)

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,...,rawcensustractandblock,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,county
584,17052889,0.055619,2017-01-01,1.0,2,1465,6111,34449266.0,-119281531.0,12647,...,6.111001e+07,13091,97099,1967,88000,464000,376000,5672.48,61110010023006,6111


In [11]:
# splits test off, 20% of original df size, splits validate off 40% of what remains (32% of original df size)
train, validate, test = wrangle.split_zillow(z_df)
print("train observations: ", train.shape[0])
print("validate observations: ", validate.shape[0])
print("test observations: ", test.shape[0])

train observations:  19908
validate observations:  13272
test observations:  8296
